MY_VQ_EVAL_TEST

In [1]:
import os
import json

import torch
os.chdir("/workspace/")
from torch.utils.tensorboard import SummaryWriter
import numpy as np
import models.vqvae as vqvae
import utils.utils_model as utils_model
from dataset import dataset_TM_eval
# import utils.eval_trans as eval_trans
from options.get_eval_option import get_opt
from models.evaluator_wrapper import EvaluatorModelWrapper
import warnings
warnings.filterwarnings('ignore')

In [2]:

import clip
import numpy as np
import torch
from scipy import linalg

import visualization.plot_3d_global as plot_3d
from utils.motion_process import recover_from_ric

In [85]:
def calculate_diversity(activation, diversity_times):
    assert len(activation.shape) == 2
    assert activation.shape[0] > diversity_times
    #activation ([4650, 512])
    num_samples = activation.shape[0]
    
    # 4650개숫자중  300개를 뽑음
    first_indices = np.random.choice(num_samples, diversity_times, replace=False)
    second_indices = np.random.choice(num_samples, diversity_times, replace=False)
    
    #벡터의 norm은 원점에서 벡터까지의 거리를 나타내며, 일반적으로 유클리드 놈(Euclidean norm)이 가장 자주 사용됩니
    dist = linalg.norm(activation[first_indices] - activation[second_indices], axis=1)
    return dist.mean()


In [82]:
def calculate_frechet_distance(mu1, sigma1, mu2, sigma2, eps=1e-6):

    #실제모션의 평균
    mu1 = np.atleast_1d(mu1)
    #생성모션의 평균
    mu2 = np.atleast_1d(mu2)

    sigma1 = np.atleast_2d(sigma1)
    sigma2 = np.atleast_2d(sigma2)

    assert mu1.shape == mu2.shape, \
        'Training and test mean vectors have different lengths'
    assert sigma1.shape == sigma2.shape, \
        'Training and test covariances have different dimensions'

    diff = mu1 - mu2

    # Product might be almost singular

    #sigma1과 sigma2의 행렬곱의 제곱근 
    covmean, _ = linalg.sqrtm(sigma1.dot(sigma2), disp=False)
    if not np.isfinite(covmean).all():
        msg = ('fid calculation produces singular product; '
               'adding %s to diagonal of cov estimates') % eps
        print(msg)
        offset = np.eye(sigma1.shape[0]) * eps
        covmean = linalg.sqrtm((sigma1 + offset).dot(sigma2 + offset))

    # Numerical error might give slight imaginary component
    if np.iscomplexobj(covmean):
        if not np.allclose(np.diagonal(covmean).imag, 0, atol=1e-3):
            m = np.max(np.abs(covmean.imag))
            raise ValueError('Imaginary component {}'.format(m))
        covmean = covmean.real

    tr_covmean = np.trace(covmean)

    return (diff.dot(diff) + np.trace(sigma1)
            + np.trace(sigma2) - 2 * tr_covmean)

def calculate_activation_statistics(activations):

    mu = np.mean(activations, axis=0)

    #np.cov 함수는 주어진 데이터의 공분산 행렬(covariance matrix)을 계산하는 함수
    #rowvar=False -> 열을 변수로 
    cov = np.cov(activations, rowvar=False)
    return mu, cov

In [133]:
def euclidean_distance_matrix(matrix1, matrix2):
    """
        Params:
        -- matrix1: N1 x D          32,512
        -- matrix2: N2 x D          32,512
        Returns:
        -- dist: N1 x N2
        dist[i, j] == distance(matrix1[i], matrix2[j])
    """
    assert matrix1.shape[1] == matrix2.shape[1]
    d1 = -2 * np.dot(matrix1, matrix2.T)    # shape (num_test, num_train) => (32, 32)

    # matrix1의 각 샘플의 제곱값을 더한 후, 열이 1개인 행렬로 만듭니다. 
    d2 = np.sum(np.square(matrix1), axis=1, keepdims=True)    # shape (num_test, 1) (32,1)
    d3 = np.sum(np.square(matrix2), axis=1)     # shape (num_train, )  (32,)
    dists = np.sqrt(d1 + d2 + d3)  # broadcasting

    # (32,32)
    return dists


In [206]:
import numpy as np

mat = np.array([[2, 3, 0 ,1],
                [3, 2, 1, 0],
                [0, 1, 2, 3],
                [1, 2, 3, 0]])

size = mat.shape[0]
gt_mat = np.expand_dims(np.arange(size), 1).repeat(size, 1)

bool_mat = (mat == gt_mat)


correct_vec = False
top_k_list = []
for i in range(3):
    correct_vec = (correct_vec | bool_mat[:, i])
    print(correct_vec[:, None])
    top_k_list.append(correct_vec[:, None])
# print(top_k_list)
top_k_mat = np.concatenate(top_k_list, axis=1)
# print(top_k_mat)
#매칭이 있는 위치는 True로, 그렇지 않은 위치는 False로 표시됩니다.


print("mat:")
print(mat)
print("\ngt_mat:")
print(gt_mat)
print("\nbool_mat:")
print(bool_mat)
print("\ncorrect_vec")
print(correct_vec)
print("\ntop_k_list")
print(top_k_list)
print("\ntop_k_mat")
print(top_k_mat)
print("\ntop_k_mat")
print(top_k_mat.sum(axis=0))

[[False]
 [False]
 [False]
 [False]]
[[False]
 [False]
 [False]
 [False]]
[[ True]
 [ True]
 [ True]
 [ True]]
mat:
[[2 3 0 1]
 [3 2 1 0]
 [0 1 2 3]
 [1 2 3 0]]

gt_mat:
[[0 0 0 0]
 [1 1 1 1]
 [2 2 2 2]
 [3 3 3 3]]

bool_mat:
[[False False  True False]
 [False False  True False]
 [False False  True False]
 [False False  True False]]

correct_vec
[ True  True  True  True]

top_k_list
[array([[False],
       [False],
       [False],
       [False]]), array([[False],
       [False],
       [False],
       [False]]), array([[ True],
       [ True],
       [ True],
       [ True]])]

top_k_mat
[[False False  True]
 [False False  True]
 [False False  True]
 [False False  True]]

top_k_mat
[0 0 4]


In [207]:
def calculate_top_k(mat, top_k):
    #mat (32, 32)
    size = mat.shape[0] #32

    # mat은 정렬된 index의 값 
    gt_mat = np.expand_dims(np.arange(size), 1).repeat(size, 1)
    bool_mat = (mat == gt_mat)
    #같은면 True

    correct_vec = False
    top_k_list = []
    for i in range(top_k):
        # print(correct_vec, bool_mat[:, i])
        correct_vec = (correct_vec | bool_mat[:, i])
        top_k_list.append(correct_vec[:, None])
    top_k_mat = np.concatenate(top_k_list, axis=1)
    # print(top_k_mat)
    #매칭이 있는 위치는 True로, 그렇지 않은 위치는 False로 표시됩니다.
    return top_k_mat


In [213]:
def calculate_R_precision(embedding1, embedding2, top_k, sum_all=False):
    #embedding1 = embedding text,  embedding2 = embedding motion
    dist_mat = euclidean_distance_matrix(embedding1, embedding2)
    
    # 대각선의 요소들을 합한 값(MM-Dist) => 텍스트 feature와 text feauture 간의 거리를더함
    matching_score = dist_mat.trace()

    # 행을 기준으로 정렬(axis=1) 오름차순 (낮을수록 거리가 가까운거)
    argmax = np.argsort(dist_mat, axis=1)
    print(argmax)
    top_k_mat = calculate_top_k(argmax, top_k)

    #흠 top1에 각 32 모션과 매칭된게 들어있는지 
    #   top2에 안에 각 32 모션에 매칭된 것이 있는지 
    #   top3에 각 32개 모션에 매칭 된 것이 있는지 계산하는거
    if sum_all:
        return top_k_mat.sum(axis=0), matching_score
    else:
        return top_k_mat, matching_score

In [209]:
@torch.no_grad()        
def evaluation_vqvae(out_dir, val_loader, net, logger, writer, nb_iter, best_fid, best_iter, best_div, best_top1, best_top2, best_top3, best_matching, eval_wrapper, draw = True, save = True, savegif=False, savenpy=False) : 
    net.eval()
    nb_sample = 0
    
    draw_org = []
    draw_pred = []
    draw_text = []


    motion_annotation_list = []
    motion_pred_list = []

    R_precision_real = 0
    R_precision = 0

    nb_sample = 0
    matching_score_real = 0
    matching_score_pred = 0
    for batch in val_loader:
        word_embeddings, pos_one_hots, caption, sent_len, motion, m_length, token, name = batch


        # ([32, 196, 263])
        motion = motion.cuda()
        et, em = eval_wrapper.get_co_embeddings(word_embeddings, pos_one_hots, sent_len, motion, m_length)
        # et -> ([32, 512]) , em ->[32, 512]
        
        bs, seq = motion.shape[0], motion.shape[1]
        #bs = 32, seq = 196

        num_joints = 21 if motion.shape[-1] == 251 else 22
        
        #예측한것들 모아두기 위한 초기화 작업
        pred_pose_eval = torch.zeros((bs, seq, motion.shape[-1])).cuda()

        for i in range(bs):
            pose = val_loader.dataset.inv_transform(motion[i:i+1, :m_length[i], :].detach().cpu().numpy())
            pose_xyz = recover_from_ric(torch.from_numpy(pose).float().cuda(), num_joints)
            #pose_xyz = (1, m_length, 22 = num_joint, 3)
            
            pred_pose, loss_commit, perplexity = net(motion[i:i+1, :m_length[i]])
            # pred_pose = (1, frame, 263)
            # motion[i:i+1, :m_length[i]] => (1, m_length[i], 263)

            pred_denorm = val_loader.dataset.inv_transform(pred_pose.detach().cpu().numpy())
            pred_xyz = recover_from_ric(torch.from_numpy(pred_denorm).float().cuda(), num_joints)
            #pose_xyz = (1, m_length, 22 = num_joint, 3)


            # if savenpy:
            #     np.save(os.path.join(out_dir + "/npy_file/" , name[i]+'_gt.npy'), pose_xyz[:, :m_length[i]].cpu().numpy())
            #     np.save(os.path.join(out_dir + "/npy_file/", name[i]+'_pred.npy'), pred_xyz.detach().cpu().numpy())
            
            # 예측한걸듯을 pred_pose_eval기에 저장 
            pred_pose_eval[i:i+1,:m_length[i],:] = pred_pose


            # if i < min(4, bs):
            #     draw_org.append(pose_xyz)
            #     draw_pred.append(pred_xyz)
            #     draw_text.append(caption[i])

        et_pred, em_pred = eval_wrapper.get_co_embeddings(word_embeddings, pos_one_hots, sent_len, pred_pose_eval, m_length)
        # et_pred -> ([32, 512]) , em_pred ->[32, 512]
        motion_pred_list.append(em_pred)
        motion_annotation_list.append(em)
        
        # temp_R, temp_match = calculate_R_precision(et.cpu().numpy(), em.cpu().numpy(), top_k=3, sum_all=True)
        # R_precision_real += temp_R
        # matching_score_real += temp_match
        temp_R, temp_match = calculate_R_precision(et_pred.cpu().numpy(), em_pred.cpu().numpy(), top_k=3, sum_all=True)
        print(temp_R)
        R_precision += temp_R
        matching_score_pred += temp_match

        nb_sample += bs
        # nb sample = 145

    motion_annotation_np = torch.cat(motion_annotation_list, dim=0).cpu().numpy()
    # motion_annotation_np = ([4640,512])

    motion_pred_np = torch.cat(motion_pred_list, dim=0).cpu().numpy()
    # motion_pred_np = ([4640,512])

    gt_mu, gt_cov  = calculate_activation_statistics(motion_annotation_np)
    mu, cov= calculate_activation_statistics(motion_pred_np)

    diversity_real = calculate_diversity(motion_annotation_np, 300 if nb_sample > 300 else 100)
    diversity = calculate_diversity(motion_pred_np, 300 if nb_sample > 300 else 100)

   # R_precision_real = R_precision_real / nb_sample
    R_precision = R_precision / nb_sample
    print(R_precision) 
    # matching_score_real = matching_score_real / nb_sample
    # matching_score_pred = matching_score_pred / nb_sample

    fid = calculate_frechet_distance(gt_mu, gt_cov, mu, cov)

    # msg = f"--> \t Eva. Iter {nb_iter} :, FID. {fid:.4f}, Diversity Real. {diversity_real:.4f}, Diversity. {diversity:.4f}, R_precision_real. {R_precision_real}, R_precision. {R_precision}, matching_score_real. {matching_score_real}, matching_score_pred. {matching_score_pred}"
    # logger.info(msg)
    
    # if draw:
    #     writer.add_scalar('./Test/FID', fid, nb_iter)
    #     writer.add_scalar('./Test/Diversity', diversity, nb_iter)
    #     writer.add_scalar('./Test/top1', R_precision[0], nb_iter)
    #     writer.add_scalar('./Test/top2', R_precision[1], nb_iter)
    #     writer.add_scalar('./Test/top3', R_precision[2], nb_iter)
    #     writer.add_scalar('./Test/matching_score', matching_score_pred, nb_iter)

    
        # if nb_iter % 5000 == 0 : 
        #     for ii in range(4):
        #         tensorborad_add_video_xyz(writer, draw_org[ii], nb_iter, tag='./Vis/org_eval'+str(ii), nb_vis=1, title_batch=[draw_text[ii]], outname=[os.path.join(out_dir, 'gt'+str(ii)+'.gif')] if savegif else None)
            
        # if nb_iter % 5000 == 0 : 
        #     for ii in range(4):
        #         tensorborad_add_video_xyz(writer, draw_pred[ii], nb_iter, tag='./Vis/pred_eval'+str(ii), nb_vis=1, title_batch=[draw_text[ii]], outname=[os.path.join(out_dir, 'pred'+str(ii)+'.gif')] if savegif else None)   

    
    # if fid < best_fid : 
    #     msg = f"--> --> \t FID Improved from {best_fid:.5f} to {fid:.5f} !!!"
    #     logger.info(msg)
    #     best_fid, best_iter = fid, nb_iter
    #     if save:
    #         torch.save({'net' : net.state_dict()}, os.path.join(out_dir, 'net_best_fid.pth'))

    # if abs(diversity_real - diversity) < abs(diversity_real - best_div) : 
    #     msg = f"--> --> \t Diversity Improved from {best_div:.5f} to {diversity:.5f} !!!"
    #     logger.info(msg)
    #     best_div = diversity
    #     if save:
    #         torch.save({'net' : net.state_dict()}, os.path.join(out_dir, 'net_best_div.pth'))

    if R_precision[0] > best_top1 : 
    #     msg = f"--> --> \t Top1 Improved from {best_top1:.4f} to {R_precision[0]:.4f} !!!"
    #     logger.info(msg)
        best_top1 = R_precision[0]
    #     if save:
    #         torch.save({'net' : net.state_dict()}, os.path.join(out_dir, 'net_best_top1.pth'))

    if R_precision[1] > best_top2 : 
    #     msg = f"--> --> \t Top2 Improved from {best_top2:.4f} to {R_precision[1]:.4f} !!!"
    #     logger.info(msg)
        best_top2 = R_precision[1]
    
    if R_precision[2] > best_top3 : 
    #     msg = f"--> --> \t Top3 Improved from {best_top3:.4f} to {R_precision[2]:.4f} !!!"
    #     logger.info(msg)
        best_top3 = R_precision[2]
    
    # if matching_score_pred < best_matching : 
    #     msg = f"--> --> \t matching_score Improved from {best_matching:.5f} to {matching_score_pred:.5f} !!!"
    #     logger.info(msg)
    #     best_matching = matching_score_pred
    #     if save:
    #         torch.save({'net' : net.state_dict()}, os.path.join(out_dir, 'net_best_matching.pth'))

    # if save:
    #     torch.save({'net' : net.state_dict()}, os.path.join(out_dir, 'net_last.pth'))

    # net.train()best_fid, best_iter, 
    return best_fid, best_div ,best_top1, best_top2, best_top3 ,best_matching, writer, logger


In [5]:
import easydict

opt = easydict.EasyDict({
    "dataname": "t2m",
    "batch_size": 256,
    "window_size": 64,
    "total_iter": 300000,
    "warm_up_iter": 1000,
    "lr": 2e-4,
    "lr_scheduler": [200000],#[50000, 400000],
    "gamma": 0.05,
    "weight_decay": 0.0,
    "commit": 0.02,
    "loss_vel": 0.5,
    "recons_loss": "l1_smooth",
    "code_dim": 512,
    "nb_code": 512,
    "mu": 0.99,
    "down_t": 2,
    "stride_t": 2,
    "width": 512,
    "depth": 3,
    "dilation_growth_rate": 3,
    "output_emb_width": 512,
    "vq_act": "relu",
    "vq_norm": None,
    "quantizer": "ema_reset",
    "beta": 1.0,
    "resume_pth": None,
    "resume_gpt": None,
    "out_dir": "output_vqfinal/TEST_VQVAE/eval",
    "results_dir": "visual_results/",
    "visual_name": "baseline",
    "exp_name": "TEST_VQVAE/eval",
    "print_iter": 200,
    "eval_iter": 1000,
    "seed": 123,
    "vis_gt": False,
    "nb_vis": 20,
    "nb_joints" : 0,
    "resume_pth" : "output_vqfinal/VQ-VAE/eval/net_last.pth"
})

In [6]:
from utils.word_vectorizer import WordVectorizer
w_vectorizer = WordVectorizer('./glove', 'our_vab')


In [7]:
opt.nb_joints = 21 if opt.dataname == 'kit' else 22

val_loader = dataset_TM_eval.DATALoader(opt.dataname, True, 32, w_vectorizer, unit_length=2**opt.down_t)

##### ---- Network ---- #####
net = vqvae.HumanVQVAE(opt, ## use opt to define different parameters in different quantizers
                       opt.nb_code,
                       opt.code_dim,
                       opt.output_emb_width,
                       opt.down_t,
                       opt.stride_t,
                       opt.width,
                       opt.depth,
                       opt.dilation_growth_rate,
                       opt.vq_act,
                       opt.vq_norm)

100%|██████████| 4384/4384 [00:00<00:00, 4975.81it/s]


Pointer Pointing at 0


In [9]:
if opt.resume_pth : 
    print("1")
    ckpt = torch.load(opt.resume_pth, map_location='cpu')
    net.load_state_dict(ckpt['net'], strict=True)
net.train()
net.cuda()

1


HumanVQVAE(
  (vqvae): VQVAE_251(
    (encoder): Encoder(
      (model): Sequential(
        (0): Conv1d(263, 512, kernel_size=(3,), stride=(1,), padding=(1,))
        (1): ReLU()
        (2): Sequential(
          (0): Conv1d(512, 512, kernel_size=(4,), stride=(2,), padding=(1,))
          (1): Resnet1D(
            (model): Sequential(
              (0): ResConv1DBlock(
                (norm1): Identity()
                (norm2): Identity()
                (activation1): ReLU()
                (activation2): ReLU()
                (conv1): Conv1d(512, 512, kernel_size=(3,), stride=(1,), padding=(9,), dilation=(9,))
                (conv2): Conv1d(512, 512, kernel_size=(1,), stride=(1,))
              )
              (1): ResConv1DBlock(
                (norm1): Identity()
                (norm2): Identity()
                (activation1): ReLU()
                (activation2): ReLU()
                (conv1): Conv1d(512, 512, kernel_size=(3,), stride=(1,), padding=(3,), dilation=(3,))
 

In [212]:
fid = []
div = []
top1 = []
top2 = []
top3 = []
matching = []
repeat_time = 1
for i in range(repeat_time):
    #     best_fid, best_iter,  , best_matching, writer, logger 
    best_fid , best_div, best_top1, best_top2, best_top3 = evaluation_vqvae(opt.out_dir, val_loader, net, None, None, 0, best_fid=1000, best_iter=0, best_div=100, best_top1=0, best_top2=0, best_top3=0, best_matching=100, eval_wrapper=eval_wrapper, draw=False, savegif=False, save=False, savenpy=False)
    fid.append(best_fid)
    div.append(best_div)
    top1.append(best_top1)
    top2.append(best_top2)
    top3.append(best_top3)
    # matching.append(best_matching)
print('final result:')
print('fid: ', sum(fid)/repeat_time)

# print('div: ', sum(div)/repeat_time)
print('top1: ', sum(top1)/repeat_time)
print('top2: ', sum(top2)/repeat_time)
print('top3: ', sum(top3)/repeat_time)
# print('matching: ', sum(matching)/repeat_time)
# fid = np.array(fid)
# div = np.array(div)
# top1 = np.array(top1)
# top2 = np.array(top2)
# top3 = np.array(top3)
# matching = np.array(matching)
# msg_final = f"FID. {np.mean(fid):.3f}, conf. {np.std(fid)*1.96/np.sqrt(repeat_time):.3f}, Diversity. {np.mean(div):.3f}, conf. {np.std(div)*1.96/np.sqrt(repeat_time):.3f}, TOP1. {np.mean(top1):.3f}, conf. {np.std(top1)*1.96/np.sqrt(repeat_time):.3f}, TOP2. {np.mean(top2):.3f}, conf. {np.std(top2)*1.96/np.sqrt(repeat_time):.3f}, TOP3. {np.mean(top3):.3f}, conf. {np.std(top3)*1.96/np.sqrt(repeat_time):.3f}, Matching. {np.mean(matching):.3f}, conf. {np.std(matching)*1.96/np.sqrt(repeat_time):.3f}"
# logger.info(msg_final)

[[29  5 13 ... 24 23 18]
 [ 7  6  1 ... 11 23 18]
 [ 0 12  9 ... 11 18 24]
 ...
 [10  5 29 ... 27 24  4]
 [20 27 22 ...  4 29 14]
 [26 31  3 ... 18 24 11]]
[[ 0  0  0 ...  0  0  0]
 [ 1  1  1 ...  1  1  1]
 [ 2  2  2 ...  2  2  2]
 ...
 [29 29 29 ... 29 29 29]
 [30 30 30 ... 30 30 30]
 [31 31 31 ... 31 31 31]]
[[ 0  0  0 ...  0  0  0]
 [ 1  1  1 ...  1  1  1]
 [ 2  2  2 ...  2  2  2]
 ...
 [29 29 29 ... 29 29 29]
 [30 30 30 ... 30 30 30]
 [31 31 31 ... 31 31 31]]
[15 21 27]
[[ 0 12 13 ... 20  6  2]
 [ 5  2  3 ... 24  9 30]
 [17 22  4 ... 30 29 31]
 ...
 [29 31 25 ... 19 20  6]
 [ 7  9 30 ... 31  3 29]
 [15 31 16 ... 19  6 20]]
[[ 0  0  0 ...  0  0  0]
 [ 1  1  1 ...  1  1  1]
 [ 2  2  2 ...  2  2  2]
 ...
 [29 29 29 ... 29 29 29]
 [30 30 30 ... 30 30 30]
 [31 31 31 ... 31 31 31]]
[[ 0  0  0 ...  0  0  0]
 [ 1  1  1 ...  1  1  1]
 [ 2  2  2 ...  2  2  2]
 ...
 [29 29 29 ... 29 29 29]
 [30 30 30 ... 30 30 30]
 [31 31 31 ... 31 31 31]]
[11 17 22]
[[28 18  9 ... 25 19 21]
 [18  6  1 ... 19

KeyboardInterrupt: 